# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [45]:
!pip install ultralytics

In [46]:
import cv2
import numpy as np
from ultralytics import YOLO

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
model = YOLO("/content/drive/MyDrive/T5-TWI/Week 6/6- Introduction to computer vision/5- Weekly Project (Sunday)/Dataset/YOLO_Model.pt")

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [62]:
video_path = 'input_video.mp4'
cap = cv2.VideoCapture('/content/Manal.mp4')

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [63]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # for the width
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # for the height
fps = int(cap.get(cv2.CAP_PROP_FPS))  # for the fps in each frame

In [64]:
print(width ,height ,fps) # Checking for sizes for each

1920 1080 29


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [65]:
output_path = '/content/output_video_with_predictions.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

In [66]:
ret,frame =cap.read()
results =model(frame)


0: 384x640 2 buss, 90 cars, 2 trucks, 2 vans, 64.9ms
Speed: 3.1ms preprocess, 64.9ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)


## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [67]:
from google.colab.patches import cv2_imshow
 #(ROIs)
top_roi = (0, 0, width, 100)  # Top exit
bottom_roi = (0, height - 100, width, height)  # Bottom exit
left_roi = (0, 0, 100, height)  # Left exit
right_roi = (width - 100, 0, width, height)  # Right exit

# Draw the ROIs on the frame
cv2.rectangle(frame, (top_roi[0], top_roi[1]), (top_roi[2], top_roi[3]), (0, 255, 0), 2)
cv2.rectangle(frame, (bottom_roi[0], bottom_roi[1]), (bottom_roi[2], bottom_roi[3]), (0, 255, 0), 2)
cv2.rectangle(frame, (left_roi[0], left_roi[1]), (left_roi[2], left_roi[3]), (0, 255, 0), 2)
cv2.rectangle(frame, (right_roi[0], right_roi[1]), (right_roi[2], right_roi[3]), (0, 255, 0), 2)



array([[[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [ 34,  75,  69],
        ...,
        [ 55,  51,  47],
        [ 55,  51,  47],
        [  0, 255,   0]],

       ...,

       [[  0, 255,   0],
        [  0, 255,   0],
        [ 99,  87, 163],
        ...,
        [108, 117, 123],
        [108, 117, 123],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [112, 100, 164],
        ...,
        [110, 119, 125],
        [108, 117, 123],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]]

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [68]:
# ROIs in all direct*
top_roi = (0, 0, width, 50)  # Top exit
bottom_roi = (0, height - 50, width, height)  # Bottom exit
left_roi = (0, 0, 50, height)  # Left exit
right_roi = (width - 50, 0, width, height)  # Right exit

# count each vehicles exiting in each street
top_count = 0
bottom_count = 0
left_count = 0
right_count = 0

# Process each frame in the video
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLO model on the frame
        results = model(frame)

        # Draw bounding boxes and display information
        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy()

            for i in range(len(boxes)):
                x1, y1, x2, y2 = boxes[i]
                confidence = confidences[i]
                class_id = classes[i]

                # Calculate center of the bounding box
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)

                # Draw bounding box
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

                # Display confidence score, object ID, and class ID
                text = f"Conf: {confidence:.2f}, Class: {class_id}"
                cv2.putText(frame, text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Display center of the bounding box
                cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)

                # Check if the vehicle exited from any street
                if center_y < top_roi[3]:  # Top exit
                    top_count += 1
                if center_y > bottom_roi[1]:  # Bottom exit
                    bottom_count += 1
                if center_x < left_roi[2]:  # Left exit
                    left_count += 1
                if center_x > right_roi[0]:  # Right exit
                    right_count += 1

        # Draw ROIs on the frame
        cv2.rectangle(frame, (top_roi[0], top_roi[1]), (top_roi[2], top_roi[3]), (0, 255, 0), 2)
        cv2.rectangle(frame, (bottom_roi[0], bottom_roi[1]), (bottom_roi[2], bottom_roi[3]), (0, 255, 0), 2)
        cv2.rectangle(frame, (left_roi[0], left_roi[1]), (left_roi[2], left_roi[3]), (0, 255, 0), 2)
        cv2.rectangle(frame, (right_roi[0], right_roi[1]), (right_roi[2], right_roi[3]), (0, 255, 0), 2)

        # Display the count of vehicles exiting each street
        cv2.putText(frame, f"Top: {top_count}", (10, 30), cv2.FONT_ITALIC, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Bottom: {bottom_count}", (10, 60), cv2.FONT_ITALIC, 1, (0, 0, 255), 2)
        cv2.putText(frame, f"Left: {left_count}", (10, 90), cv2.FONT_ITALIC, 1, (0, 0, 255), 2)
        cv2.putText(frame, f"Right: {right_count}", (10, 120), cv2.FONT_ITALIC, 1, (0, 0, 255), 2)

        # Display your name
        cv2.putText(frame, "Eng.Manal", (10, height - 20), cv2.FONT_ITALIC, 1, (0, 255, 0), 2)

        # Write the frame to the output video
        out.write(frame)

    else:
        break


    out.write(frame)
cap.release()
out.release()
print(f"Video saved to {output_path}")

#cv2.destroyAllWindows()


0: 384x640 2 buss, 92 cars, 2 trucks, 3 vans, 62.9ms
Speed: 3.3ms preprocess, 62.9ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 58.7ms
Speed: 3.4ms preprocess, 58.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 54.0ms
Speed: 3.7ms preprocess, 54.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 94 cars, 2 trucks, 2 vans, 53.5ms
Speed: 3.4ms preprocess, 53.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 53.4ms
Speed: 2.9ms preprocess, 53.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 43.2ms
Speed: 3.2ms preprocess, 43.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 42.0ms
Speed: 3.0ms preprocess, 42.0ms inferenc

## Save and Submit Your Work

Submit both the notebook and the output video

For the video I tried to apply changed but keep to show the old version ,all my apologies for this :(

Vid link:

https://drive.google.com/file/d/1s8QAtXUaciMsmbwp8qSUpZ00k_S6h_aW/view?usp=sharing
